## Report Notebook

This is just a notebook to compare the results of the generative models on several datasets, saving the results on wandb as tables

Models to compare:

VAE    <br>
VQVAE  <br>
VQVAE2 <br>

GAN    <br>
WGAN   <br>
PCGAN  <br>

VAEGAN [ ] <br> 
VQGAN  [ ] <br> 

NVAE   [ ] <br> 




In [1]:
import matplotlib.pyplot as plt
import wandb
import numpy as np
import os
from os.path import join as opj
from classes.VAE import VAE
from classes.VQVAE import VQVAE, VQVAE2
from classes.GAN import GAN

import tensorflow as tf
from tensorflow import keras
from tensorflow.data import AUTOTUNE
from imutils import paths


In [2]:
wandb.login()

BS=128

#set the second GPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"
print(os.environ.get("CUDA_VISIBLE_DEVICES"))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: matteoferrante (use `wandb login --relogin` to force relogin)


0


### Models path

In [4]:
vae_path="models/vae_celeba"
vqvae_path="models/vqvae_celeba"
vqvae2_path="models/vqvae2_celeba"

In [6]:
print(f"vae {os.listdir(vae_path)}")
print(f"\n vqvae {os.listdir(vqvae_path)}")
print(f"\n vqvae2 {os.listdir(vqvae2_path)}")


vae ['vae_model.png', 'encoder_weights.h5', 'decoder_weights.h5']

 vqvae ['vqvae_model.png', 'vqvae_encoder_model.png', 'vqvae_decoder_model.png', 'vq_vae_encoder.h5', 'vq_vae_decoder.h5', 'vq_vae_embeddings.npy']

 vqvae2 ['vqvae2_encoder_bottom.png', 'vqvae2_encoder_top.png', 'vqvae2_decoder.png', 'vqvae2_conditional_bottom.png', 'vqvae2_model.png', 'vq_vae2_encoder_b.h5', 'vq_vae2_encoder_t.h5', 'vq_vae2_encoder_conditional_bottom.h5', 'vq_vae2_decoder.h5']


# CelebA comparison

In [5]:
#images_dir=r"C:\Users\matte\Dataset\tor_vergata\Dataset\Img\img_align_celeba" #local
images_dir=r"/home/matteo/NeuroGEN/Dataset/Img/img_align_celeba"



In [6]:
def load_images(imagePath):
    # read the image from disk, decode it, resize it, and scale the
    # pixels intensities to the range [0, 1]
    image = tf.io.read_file(imagePath)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (128, 128)) / 255.0

    #eventually load other information like attributes here
    
    # return the image and the extra info
    
    
    return image

In [ ]:
print("[INFO] loading image paths...")
imagePaths = list(paths.list_images(images_dir))


train_len=int(0.8*len(imagePaths))
val_len=int(0.1*len(imagePaths))
test_len=int(0.1*len(imagePaths))

train_imgs=imagePaths[:train_len]                                #      80% for training
val_imgs=imagePaths[train_len:train_len+val_len]                 #      10% for validation
test_imgs=imagePaths[train_len+val_len:]                         #      10% for testing

print(f"[TRAINING]\t {len(train_imgs)}\n[VALIDATION]\t {len(val_imgs)}\n[TEST]\t\t {len(test_imgs)}")

[INFO] loading image paths...


In [ ]:
#TRAINING 

train_dataset = tf.data.Dataset.from_tensor_slices(train_imgs)
train_dataset = (train_dataset
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .repeat()
    .batch(BS)
    .shuffle(1024)
    .prefetch(AUTOTUNE)
)

ts=len(train_imgs)//BS

##VALIDATION

val_dataset = tf.data.Dataset.from_tensor_slices(val_imgs)
val_dataset = (val_dataset
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .repeat()
    .shuffle(1024)
    .batch(BS)
    .prefetch(AUTOTUNE)
)

vs=len(val_imgs)//BS

## TEST

test_dataset = tf.data.Dataset.from_tensor_slices(test_imgs)
test_dataset = (test_dataset
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .shuffle(1024)
    .batch(BS)
    .prefetch(AUTOTUNE)
)

## VAE

In [ ]:
encoder_architecture=[(0,64),(0,128),(0,256),(0,384),(0,512)]
decoder_architecture=[(0,512),(0,384),(0,256),(0,128),(0,64)]

vae=VAE((128,128,3),
      latent_dim=512,
      encoder_architecture=encoder_architecture,
      decoder_architecture=decoder_architecture,
      output_channels=3)

vae.encoder.load_weights(opj(vae_path,"encoder_weights.h5"))
vae.decoder.load_weights(opj(vae_path,"decoder_weights.h5"))

## VQVAE

In [ ]:
## re train the model and save the weights!

## VQVAE2

In [ ]:
input_shape=(128,128,3)
latent_dim=256
num_embeddings=1024

vqvae2=VQVAE2(input_shape,latent_dim=latent_dim,num_embeddings=num_embeddings,train_variance=4,n_res_channel=latent_dim,channels=128)
vqvae2.encoder_b.load_weights(opj(vqvae2_path,"vq_vae2_encoder_b.h5"))
vqvae2.encoder_t.load_weights(opj(vqvae2_path,"vq_vae2_encoder_t.h5"))
vqvae2.conditional_bottom.load_weights(opj(vqvae2_path,'vq_vae2_encoder_conditional_bottom.h5'))
vqvae2.decoder.load_weights(opj(vqvae2_path,'vq_vae2_decoder.h5'))

## load also the codebooks!


In [ ]:
x=test_dataset.take(1)

In [ ]:
y=vqvae2.predict(x)

In [ ]:
plt.imshow(y[0])